#### This notebook contains routines for constructing a database from raw data downloaded from cathdb.info/download and CATH-API



In [1]:
from tst.utils_db import *


### Specify the version tag of your session...
### Create a new tag if not existing
v_curr = verify_version('putative')

print(v_curr)



putative


In [27]:
#### Debug failed attempts of domain_stat_fill()

# # dname = flst[0]
# dname = '3rfwA01'
# d = domain.objects.get(domain_id = dname)
# try:
#     domain_stat_fill(d,env = env) 
# except Exception as e:
#     flst.append(d.domain_id)
#     print("Unable to process %s for %s" % (d.domain_id, str(e)))

In [ ]:
### Attempt to calculate structure-based statistics for a collection of domains

#from tst.utils_db import *

dset = domain.objects.filter( classification__version = v_curr )
# dset = dset.annotate(version = 'classification__version')

#### Initialise env for modeller, and specify the path to search for pdbs
env =init_env()

lst = dset
imax = len(lst)
flst = []

for i,d in enumerate(lst):
    try:
        domain_stat_fill(d,env = env) 
    except Exception as e:
        flst.append(d.domain_id)
        print("Unable to process %s for %s" % (d.domain_id, str(e)))
    if not i % 100:
        print('%d of %d'%(i,imax))

In [2]:
#### Calculate superfamily-based statistics

from time import time
t0 = time()
from tst.utils_db import *

level_curr = level.objects.get(letter = 'H')
# node_set =  classification.objects.filter(version = v_curr)
node_set =  classification.objects.filter(version__id__lte = v_curr.id)



lst = node_set.filter(level = level_curr)

c = counter(lst, per = 100)

failcount = 0
for d in lst:
    c.count()
    with transaction.atomic():
#     if 1:
        failcount += homsf_stat_fill(d) == 0

print '%d instances failed' % failcount
print('Ended after %.4f' % (time()-t0))  # len(lst)
# homsf_stat_fill(lst[0])
# print(str(lst[0]))

0 of 6874
100 of 6874
200 of 6874
300 of 6874
400 of 6874
500 of 6874
600 of 6874
700 of 6874
800 of 6874
900 of 6874
1000 of 6874
1100 of 6874
1200 of 6874
1300 of 6874
1400 of 6874
1500 of 6874
1600 of 6874
1700 of 6874
1800 of 6874
1900 of 6874
2000 of 6874
2100 of 6874
2200 of 6874
2300 of 6874
2400 of 6874
2500 of 6874
2600 of 6874
2700 of 6874
2800 of 6874
2900 of 6874
3000 of 6874
3100 of 6874
3200 of 6874
3300 of 6874
3400 of 6874
3500 of 6874
3600 of 6874
3700 of 6874
3800 of 6874
3900 of 6874
4000 of 6874
4100 of 6874
4200 of 6874
4300 of 6874
4400 of 6874
4500 of 6874
4600 of 6874
4700 of 6874
4800 of 6874
4900 of 6874
5000 of 6874
5100 of 6874
5200 of 6874
5300 of 6874
5400 of 6874
5500 of 6874
5600 of 6874
5700 of 6874
5800 of 6874
5900 of 6874
6000 of 6874
6100 of 6874
6200 of 6874
6300 of 6874
6400 of 6874
6500 of 6874
6600 of 6874
6700 of 6874
6800 of 6874
0 instances failed
Ended after 216.3868


In [1]:
h = classification.objects.get_bytree('1.10.30.10')[0]
# print(h)
# classification.objects.get_bytree

1.10.30.10.0


In [5]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline

import numpy as np

data = np.random.randn(20, 40)

pipeline = Pipeline([
    ('pca', PCA(n_components=2)),
    ('scaling', StandardScaler()),
                    ])

pipeline.fit_transform(data)
data.shape

sset = h.classification_set.all()
(xs,ys,lbls) = zip(*sset.values_list('domain__domain_stat__atom_count','domain__domain_stat__nbpair_count','id'))
data = np.array([xs,ys])
# lbls
nd = pipeline.fit_transform(data.T)
xs = nd[:,0]
ys = nd[:,1]
# print nd.shape
# print data.shape
# scatterplot_dict(xs,ys,lbls, regress = 0, show = 1)

pip_pc_std = Pipeline([
    ('pca', PCA(n_components=2)),
    ('scaling', StandardScaler()),
                    ])
pcdata = pip_pc_std.fit_transform(data.T)
pcxs,pcys = pcdata.T
# pcy = pcdata[:,1]
print pcxs.shape
(pcxs**2+pcys**2) - np.array(sset.values_list('domain__domain_stat__maha_dist')).T
# def pc_std(data):

(235,)


In [7]:
from tst.utils import *

h = classification.objects.get_bytree('1.10.30.10')[0]
sset = h.classification_set

(xs,ys,lbls) = zip(*sset.values_list('domain__domain_stat__atom_count','domain__domain_stat__nbpair_count','id'))
# results = PCA(data)

In [2]:
from tst.utils_db import *
h = classification.objects.get_bytree("2.40.50.140")[0]
print homsf_stat_fill(h)

1


In [1]:
### plotting mahalanobis distance
# del scatterplot_dict.__module__
from tst.utils import *

h = classification.objects.get_bytree('1.10.30.10')[0]
sset = h.classification_set

(xs,ys,lbls) = zip(*sset.values_list('parent__node_stat__Rsq_NBcount_Acount','domain__domain_stat__maha_dist','id'))

# lsts = zip(*sset.values_list('parent__node_stat__Rsq_NBcount_Acount','domain__domain_stat__maha_dist','id'))
# scatterplot_dict(*lsts, regress=0, show = 1)
scatterplot_dict(xs,ys,lbls, regress = 0, show = 1)


ImportError: No module named tst.utils

In [40]:
### Benchmarking object retrival in tree-structured database
### Compared naive search ".get()"  against ".get_bytree()"
### Consider using Django-mptt instead of writing custom functions

# node = '1.10.40.30.0'
# node = '1.10.60.10.3'
# levels=['Class',
# 'arch',
# 'topo',
# 'homsf',
# 's35',
# 's60',
# 's95',
# 's100'];

node = '2.40.500.20.0'
qdict = dict(zip(levels,node.split('.')))
c0 = None
# c0 = classification.objects.get(**dict(zip(levels,node.split('.')[:2]+['0'])))
from time import time

n = 2
t0=time()
for i in range(n):
    try:
        c1 = classification.objects.get(**qdict)
    except:
        c1 = [];
# print
t1=time()
for i in range(n):
    c2 = classification.objects.get_bytree(node,qnode = c0)[0]
t2=time()    
print('start at',c0)
print(c1)
print(t1-t0)
print(c2)
print(t2-t1)

('start at', None)
[]
0.000247001647949
2.40.500.20.0
0.0172209739685


In [5]:
#### Fetch data from cathdb.info

import urllib2

from StringIO import StringIO
import gzip


putative_s35_url = 'http://download.cathdb.info/cath/releases/daily-release/newest/cath-b-s35-newest.gz'

request = urllib2.Request( putative_s35_url )
# request.add_header('Accept-encoding', 'gzip')
response = urllib2.urlopen(request)
# if response.info().get('Content-Encoding') == 'gz':
if 1:
    buf = StringIO(response.read())
    f = gzip.GzipFile(fileobj=buf)
    data = f.read()# len(f)
# type(f)
# help(f)
# lines = data.splitlines()

In [5]:
node = '2.40.500.0'
qdict = dict(zip(levels,node.split('.')))
c1 = classification.objects.filter(**qdict)
for c in c1:
# c1
#     print(c)
    if c.classification_set.all().exists():
        print(c.classification_set.all())
    else:
        c.delete()
        print('no domain')

<QuerySet [<classification: 2.40.500.10.0>]>


In [2]:
# c2.classification_set.all()
levels

[None, 'root', 'Class', 'arch', 'topo', 'homsf', 's35', 's60', 's95', 's100']

In [4]:
# c = c2.classification_set.all()[0]

from copy import copy
# c=copy(c2)
# c.pk = None
# c.level = None
ndict = c2.node_dict()
lv = c2.level.id
c = classification.objects.create(level_id=lv+1,version_id = 2,**ndict)
# c.level = level.objects.get(id = c2.level_id+1);
c.save()
print(c2.level_id,c.level_id)
print(c2.level,c.level)

c.delete()
# c2.level_id

NameError: name 'c2' is not defined

In [10]:
### Codes here before goes into utils_db.py
### parse_domain add domain/s35 to the existing hierarchy 

from domutil.util import *
from copy import copy
def verify_version(ver):
    #### Check whether this version is recorded in 'version' table
#     try:
    vset = version.objects.filter(name=ver)
    if vset.count() > 1:
        raise Exception,'multiple version with name %s'%(ver)
    elif not vset.exists():
        v = version(name=ver)
        v.save()
    else:
        v = vset[0]
    return v

verify_version('putative')

def parse_domain(line):
    global cc
    lst = line.split()
    
    domain_id = lst[0]
    ver = 'putative'
#     ver = lst[1]
#     node_str = lst[2]
    homsf_str = lst[1]    
    chopping = lst[2]
    
    ### verify version
    v = verify_version(ver)

    #### Check whether this node exists in 'classification' table
#     (node,success) = classification.objects.get_bytree(node_str)
    (node,success) = classification.objects.get_bytree(homsf_str)
    
    ### recursively  create new superfamily if not existing
#     print(levels)
    if not success: 
        lv = node.level.id
        lst = [int(x) for x in homsf_str.split('.')]
        lst = [0,0] + lst
        while lv != len(lst)-1:
            
            lv += 1
            
            ndict = node.node_dict()
            ndict[levels[lv]] = lst[lv]
            
            node = classification.objects.create(level_id = lv,
                                                 version = v,
                                                 parent = node,
                                                 **ndict)
            node.save()
            
#             print('created %s for %s'%(str(node),homsf_str))
            cc+=1
            
    ####
    conflict = 0
    try:
        d = node.domain
        check = 1
    except:
        d = domain.objects.create(classification = node,
                                 domain_id = domain_id)
        d.save()
        check = 0

    if check:
        if str(node) in mapdict.keys():
            pass
        elif d.domain_id != domain_id:
            conflict = 1
            
            
            node = copy(node);
            node.pk = None

            lv = node.level.id
            setattr(node, levels[lv],
                    max( node.parent.classification_set.values_list( levels[lv],flat = True ) )+1
                   )
            node.version = v
            node.save()

            d = domain.objects.create(classification = node,
                                 domain_id = domain_id)
            d.save()

        else:
            pass

    return([node,conflict])
    
        
        
    #### if superfamily exists, check whether domain_id agreed
    conflict = 0;
#     print(homsf,homsf_str,success)#     print(line)

    if success:
        pass

    #### Otherwise, write conflict to file
        

In [1]:
import urllib2
from StringIO import StringIO
import gzip

#### Download s35 file from CATH

putative_s35_url = 'http://download.cathdb.info/cath/releases/daily-release/newest/cath-b-s35-newest.gz'
request = urllib2.Request( putative_s35_url )

# request.add_header('Accept-encoding', 'gzip')
response = urllib2.urlopen(request)
# if response.info().get('Content-Encoding') == 'gz':

if 1:
    buf = StringIO(response.read())
    f = gzip.GzipFile(fileobj=buf)
    data = f.read()# len(f)

mapdict = {}

In [2]:
### Writing extra domians to a temporary file

dlist = domain.objects.values_list('domain_id',flat = True)
s = set(dlist)
dlist = list(s)

lines = data.splitlines()
from domutil.util import *
newlines = [];
c = counter(lines, per = 1000)

for line in lines:
    c.count()
    if line.split()[0] not in dlist:
        newlines.append(line)
# print(newlines)

data = '\n'.join(newlines)
with open('tmp','w') as f:
    f.write(data)

In [11]:
#### Parse a CATH classification file and add nodes if necessary
cc=0
lines = data.splitlines()
c = counter(lines, per = 1000)
for line in lines:
    n,conf = parse_domain(line)
    if conf:
#         print(n.domain.domain_id,line)
        mapdict[ line.split()[1] + '.0' ] = str(n)
#         maplst +=[ (line.split()[1],str(n).rstrip('.0')) ]
#         print(line.split()[1],str(n).rstrip('.0'))
        
    else:
        pass
#         print('success',n.domain.domain_id,line)
    c.count()
#     pass
#     print(line)
    pass
print(cc)

0 of 12951
1000 of 12951
2000 of 12951
3000 of 12951
4000 of 12951
5000 of 12951
6000 of 12951
7000 of 12951
8000 of 12951
9000 of 12951
10000 of 12951
11000 of 12951
12000 of 12951
6054


In [6]:
### Parse a cath flat file 
def cathb2clf(data):
    lines = data.splitlines()
    newlines = []
    for l in lines:
#         l = '    '+ l
        lst = l.split()

        domain_id = lst[0]
        ver = 'putative'
    #     ver = lst[1]
    #     node_str = lst[2]
        node_str = lst[1]    
#         chopping = lst[2]

        nodelst = node_str.split('.') 
        nodelst += ( 9 - len(nodelst)) * ['1']
        l = [lst[0]] + nodelst + ['0','0'] 
        newlines += ['     '.join(l)]
    return('\n'.join(newlines))

# from django.conf import settings as djangoSettings
# print(djangoSettings.STATIC_ROOT)
fname = '../doc/cath-b-s35-newest.clfd'

with open(fname,'w') as f:
    d = cathb2clf(data)
    f.write(d)
# print(cathb2clf(data)[:100])
#     with open(fname) as f:
#         f.read

In [1]:
### integrity check
#### Checking for duplicated domains
dupes = ( domain.objects.values('domain_id')
    .annotate(Count('id')) 
#     .order_by()
     .filter(id__count__gt=1)
)
# dupd.count()
# dupd[0]

dupset = domain.objects.filter(domain_id__in=[item['domain_id'] for item in dupes])
print len(dupes)

d = dupset[0]
doms = d.classification.parent.classification_set.all().values_list('domain')
doms[0]

#### Delete nodes added after a version
vset = version.objects.filter(id__gte=3)
print(vset)
vset.delete()


<QuerySet [<version: putative>]>


(570L,
 {u'tst.classification': 286L,
  u'tst.domain': 283L,
  u'tst.domain_stat': 0L,
  u'tst.node_stat': 0L,
  u'tst.version': 1L})

In [11]:
### Deprecated
# # dupes[0]
# from domutil.util import *

# ct = counter(dupes)
# # for d in dupset:
# #     doms = d.classification.parent.classification_set.all().domain

# cid = [];
# did = [];
# cdel = classification.objects.none()
# ddel = domain.objects.none()


# for ele in dupes:
# # domain_id = dupes[0]['domain_id']
#     domain_id = ele['domain_id']
#     doms = dupset.filter(domain_id=domain_id)
#     vlst = doms.values_list('classification__version__id')
#     tlst = [x == min(vlst) for x in vlst]
#     for t,dom in zip(tlst,doms) :
#         ct.count()
#         if not t:
# #             print(dom.domain_id)
            
#             c = dom.classification
#             cid += [c.id]
#             did += [dom.id]
# #             cdel = cdel | c
# #             ddel = ddel | dom
# #             c.delete()
# #             dom.delete()
# # print(vlst)
# # min(doms)
# # print(dupd)

0 of 10243
100 of 10243
200 of 10243


KeyboardInterrupt: 

In [46]:
# import numpy as np 
# from domutil.util import *


# def homsf_stat_fill(h):
# # if 1:
#     if h.level.letter == 'H':
#         pass
#     else:
#         print "Node %s is not filled because it is not a homsf, but a '%s'" % (str(h),h.level.letter)
    
    
#     try:
#         nstat = h.node_stat
#     except:
#         nstat = node_stat(node = h)
#         nstat.save()
        

#     hset  = h.classification_set;
#     if hset.count()>10:
#         hset = hset.annotate(Acount=Avg("domain__domain_stat__atom_count"))
#         hset = hset.annotate(Rcount=Avg("domain__domain_stat__res_count"))
#         hset = hset.annotate(NBcount=Avg("domain__domain_stat__nbpair_count"))

#     #     hset.domain
#         l = hset.values_list('Acount','Rcount','NBcount')
#         a = np.array(l)
#     #     C = np.cov(a[:,0],a[:,1],a[:,2])
#         c = np.cov(a.T)
#         C = cov2corr(c); ## utils.cov2corr
# #         print(C.shape)
#         nstat.Rsq_NBcount_Acount = C[0,2] ** 2
#         nstat.Rsq_NBcount_Rcount = C[1,2] ** 2
#     else:
#         nstat.Rsq_NBcount_Acount = None
#         nstat.Rsq_NBcount_Rcount = None
#     try:
#         nstat.save()
#     except:
#         print nstat.__dict__
#     return h

# #     print l[:10]
# #     print l[1]
# #     print len(l)
# h = lst[2]
# homsf_stat_fill(h)